<a href="https://colab.research.google.com/github/mondalanindya/simple_binary_classification/blob/main/Binary_Classification_FMD_Pig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FMD Pig Dataset

The [FMD Pig](https://zenodo.org/record/7778284/files/FMD_Pig.zip) is a collection of pig images with and without foot and mouth diseases. So it's a dataset with two classes.

In this project, we will implement a convolutional neural network (CNN) model for classifying the images of the FMD Pig dataset. We will use PyTorch deep learning framework to complete our task.

In [ ]:
import os
if not os.path.exists('FMD_Pig.zip'):
    !wget https://zenodo.org/record/7778284/files/FMD_Pig.zip
    !unzip -q FMD_Pig.zip
    !rm FMD_Pig.zip

###Dataset and DataLoader

In the following cell, we will be defining datasets and data loaders necessary for our training. Details on datasets and dataloaders can be found in the [documentation](https://pytorch.org/vision/stable/datasets.html).

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

# Before defining datasets, lets define how images should be transformed. This is 
# because the transformations should go with the definitions of datasets. In this
# tutorial we will using simple transformations, such as (1) image to tensor, (2)
# normalization.
image_transform = Compose([Resize((112, 112)), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Once we have the transformations defined, lets define the dataset
dataset = ImageFolder('FMD_Pig', transform=image_transform)

# Once we have the dataset, lets split it into train and test set in 80% and 20%
# split
train_dataset, test_dataset = random_split(dataset=dataset, lengths=[0.8, 0.2], generator=torch.Generator().manual_seed(42))

# Once we have the datasets defined, lets define the data loaders as follows
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Example Image
Lets have a look on how images from FMD_Pig dataset looks like. Since the images are already normalized, their resolutions might have slightly changed. To visualize the original images, we should have ideally apply a reverse transformation which is avoided to keep this tutorial simple and brief.

In [ ]:
# import plot library
import matplotlib.pyplot as plt
# iterate the dataloader
_, (example_datas, labels) = next(enumerate(train_loader))
# get the first data
sample = example_datas[0]
# show the data
plt.imshow(sample.permute(1, 2, 0))
print("Label: " + str(labels[0]))

## Model
Now, we have to define trainable layers with parameters and put them inside a model. Have a look on the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module) of `nn.Module` and read more about different layers and functionalities of PyTorch there. Here we are going to implement various versions of AlexNet model and use it for classification. In this model, we are going to use the following functions or modules:

* `nn.Conv2d()`: It is a PyTorch module that applies a 2D convolution over an input signal composed of several input planes. More details are available on the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).

* `nn.Linear()`: It is a module that applies a linear transformation to the incoming data. More details can be found in its [documentation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#linear).

* `nn.ReLU()`: It is also a module that applies element-wise the rectified linear unit function. Its [documentation](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#relu) can explain more.

* `nn.Dropout()`: This module randomly zeroes some of the elements of the input tensor with probability `p`. Check the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#dropout) for more details.

One can define a model in several ways. Below, we show some of them.

In [ ]:
## We first import the pytorch nn module and optimizer
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
## Below we define the model as defined in the torchvision package and initialised 
## with pretrained weights (see pretrained=True flag)
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        from torchvision import models
        alexnet = models.alexnet(weights='IMAGENET1K_V1')
        self.features = alexnet.features
        self.avgpool = alexnet.avgpool
        self.classifier = alexnet.classifier
        # Please note how to change the last layer of the classifier for a new dataset
        # ImageNet-1K has 1000 classes, but STL-10 has 10 classes
        self.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        # Note how we are flattening the feature map, B x C x H x W -> B x C*H*W
        x = x.reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

## Initialization
Once we have the model defined, lets instantiate it and set other hyperparameters.

#### Model
We will initialize the model, transfer to the desired device and set the parameters to receive gradients.

In [ ]:
# define the model, we use AlexNet
model = AlexNet(2) # since FMD Pig dataset has 2 classes, we set num_classes = 2
# device: cuda (gpu) or cpu
device = "cuda"
# map to device
model = model.to(device) # `model.cuda()` will also do the same job
# make the parameters trainable
for param in model.parameters():
    param.requires_grad = True

#### Optimizer
For updating the parameters, PyTorch provides the package torch.optim that has most popular optimizers implemented. In this tutorial, we will be using the `torch.optim.Adam` optimizer.


In [ ]:
import torch.optim as optim
## some hyperparameters related to optimizer
learning_rate = 0.0001
weight_decay = 0.0005
# define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

## Average Meter
It is a simple class for keeping training statistics, such as losses and accuracies etc. The `.val` field usually holds the statistics for the current batch, whereas the `.avg` field hold statistics for the current epoch.

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Train and Test Functions

In [ ]:
from tqdm.notebook import tqdm
##define train function
def train(model, device, train_loader, optimizer):
    # meter
    loss = AverageMeter()
    # switch to train mode
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    for batch_idx, (data, target) in enumerate(tk0):
        # after fetching the data transfer the model to the 
        # required device, in this example the device is gpu
        # transfer to gpu can also be done by 
        # data, target = data.cuda(), target.cuda()
        data, target = data.to(device), target.to(device)  
        # compute the forward pass
        # it can also be achieved by model.forward(data)
        output = model(data) 
        # compute the loss function
        loss_this = F.cross_entropy(output, target)
        # initialize the optimizer
        optimizer.zero_grad()
        # compute the backward pass
        loss_this.backward()
        # update the parameters
        optimizer.step()
        # update the loss meter 
        loss.update(loss_this.item(), target.shape[0])
    print('Train: Average loss: {:.4f}\n'.format(loss.avg))
    return loss.avg
        
##define test function
def test(model, device, test_loader):
    # meters
    loss = AverageMeter()
    acc = AverageMeter()
    correct = 0
    # switch to test mode
    model.eval()
    for data, target in test_loader:
        # after fetching the data transfer the model to the 
        # required device, in this example the device is gpu
        # transfer to gpu can also be done by 
        # data, target = data.cuda(), target.cuda()
        data, target = data.to(device), target.to(device)  # data, target = data.cuda(), target.cuda()
        # since we dont need to backpropagate loss in testing,
        # we dont keep the gradient
        with torch.no_grad():
            # compute the forward pass
            # it can also be achieved by model.forward(data)
            output = model(data)
        # compute the loss function just for checking
        loss_this = F.cross_entropy(output, target) # sum up batch loss
        # get the index of the max log-probability
        pred = output.argmax(dim=1, keepdim=True) 
        # check which of the predictions are correct
        correct_this = pred.eq(target.view_as(pred)).sum().item()
        # accumulate the correct ones
        correct += correct_this
        # compute accuracy
        acc_this = correct_this/target.shape[0]*100.0
        # update the loss and accuracy meter 
        acc.update(acc_this, target.shape[0])
        loss.update(loss_this.item(), target.shape[0])
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        loss.avg, correct, len(test_loader.dataset), acc.avg))

## Training Loop
Training loop containing alternating train and test phase. Below we are iterating the loops 5 times, you can iterate more times.

In [ ]:
# number of epochs we decide to train
num_epoch = 5
for epoch in range(1, num_epoch + 1):
    epoch_loss = train(model, device, train_loader, optimizer)
test(model, device, test_loader)

### Summary
Show the summary of the model. It shows the number of parameters in layerwise as well as the total number of parameters. It also shows the memories required for training the model.

In [ ]:
from torchsummary import summary
summary(model, (3, 96, 96))